 # <strong>Road networks and robustness to flooding on US Atlantic and Gulf barrier islands</strong>
 ## <strong>- Download drivable road networks for the Atlantic and Gulf barrier islands -</strong>
 ### The purpose of this notebook is to download drivable road networks from OSM and retrieve the elevation of each network node using CUDEM and the exceedance probability of extreme water events associated to each node elevation.

In [ ]:
# Packages

import os
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
import rasterio
from rasterio.fill import fillnodata
import numpy as np

In [ ]:
### Set working directory

path=''
os.chdir(path)

In [ ]:
### Extract road networks from OSM, retrieve node elevation using CUDEM and calculate exceedance probability for each node (in return periods)

# Create folder if it doesn't exist
outdir= '.\\Data\\Roads2'
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
# Loop through folder containing barriers outlines (original polygons, not buffers) and use them as a mask to extract OSMnx drivable networks
rootdir = '.\\Data\\Barriers\\Barriers_AtlGulf'
extensions = ('.shp')

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier_name= file.replace('.shp', '')
            print(barrier_name)
            
            # try-statment handles exceptions that generate errors or make Python stop when networks cannot be pulled (because not all barrier islands have drivable roads)
            try: 
                # pull drivable networks using cleaned barrier polygons
                poly = gpd.read_file(file_path)
                poly_geo = poly['geometry'].iloc[0]
                poly_geo = poly_geo.buffer(0)
                G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
                N = len(G.nodes(data=True))
                print(N)
                # give to each node a new index based on integers from 0 and then add the osmid as an attribute
                osmids = list(G.nodes)
                G = nx.relabel.convert_node_labels_to_integers(G)
                osmid_values = {k:v for k, v in zip(G.nodes, osmids)}
                nx.set_node_attributes(G, osmid_values, 'initial_osmid')
                # retrieve nodes and edges in geodataframes
                nodes, edges = ox.graph_to_gdfs(G)
                # create an index for nodes in the geodataframe
                nodes['index'] = range(0, len(nodes))
                # convert nodes gdf to cudem's crs 
                nodes = nodes.to_crs("epsg:4269")
                # get coordinates of nodes
                coords = [(x,y) for x, y in zip(nodes.x, nodes.y)]
                # open corresponding CUDEM mosaic
                cudem= rasterio.open('.\\Data\\CUDEM\\CUDEM_Mosaic\\{0}.tif'.format(barrier_name))
                
                # sample the raster at every point location and store values in geodataframe
                nodes['elevation'] = [x for x in cudem.sample(coords)]
                nodes['elevation'] = nodes['elevation'].astype(float)
                
                # check if there are NA values, if true, then create a new raster with interpolated values and extract again elevation for each node:
                if nodes['elevation'].min()==-9999:
                    print('interpolation needed')
                    profile = cudem.profile
                    arr = cudem.read(1)
                    arr_filled = fillnodata(arr, mask=cudem.read_masks(1), max_search_distance=10, smoothing_iterations=0)
                    newtif_file = ('.\\Data\\CUDEM\\CUDEM_Mosaic\\{0}_interpolated.tif'.format(barrier_name))  
                    with rasterio.open(newtif_file, 'w', **profile) as dest:
                        dest.write_band(1, arr_filled)
                    cudem2= rasterio.open('.\\Data\\CUDEM\\CUDEM_Mosaic\\{0}_interpolated.tif'.format(barrier_name))
                    
                    # sample the interpolated raster at every point location and store values in geodataframe
                    nodes['elevation'] = [x for x in cudem2.sample(coords)]
                    nodes['elevation'] = nodes['elevation'].astype(float)

                # calculate exceedance probability for each network node according to their elevation 
                barrier_name= file.replace('_geo.shp', '')
                exceed = pd.read_csv(".\\Data\\Exceedance\\Probability\\{0}_Exceedance.csv".format(barrier_name), sep=",", header=0)
                for i in range(0,len(nodes)):
                    node_elev= float(nodes.elevation[i])
                    exceed_x= exceed.MaxWL
                    exceed_y= exceed.Return_Pd
                    exceedance= np.interp(node_elev, exceed_x, exceed_y)
                    nodes.at[i,'exceedance']=exceedance

                # set elevation and exceedance as attributes in the graph and save graph 
                nx.set_node_attributes(G, nodes.elevation, 'Elevations')
                nx.set_node_attributes(G, nodes.exceedance, 'Exceedance')
                ox.save_graphml(G, filepath='.\\Data\\Roads2\\{0}.graphml'.format(barrier_name))
                
            except:
                print('no drivable network')
                continue